In [7]:
import pandas as pd
import os
import re

# === CONFIG ===





files_og = [
    r"C:\Users\apagano\Downloads\srei_zcta_v101723.xlsx",
    r"C:\Users\apagano\Downloads\srei_v08042023.xlsx",
]



files = [
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024hf.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024hp.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023HP.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023POP.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023UTL.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023ENV.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023EXP.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023GEO.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023HF.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrfsn2023.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrfsn2024.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrfsn2025.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025pop.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025utl.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025env.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025exp.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025geo.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025hf.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025hp.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024pop.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024utl.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024_Feb2025.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024env.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024exp.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024geo_Feb2025.csv",]











filestesting = [

    r"...\Agreement Testing_Empl.xlsx",
]
STAGE = '@JBS_DATASETS.AREA_HEALTH_RESOURCE_FILE_AHRF.AREA_HEALTH_RESOURCE_FILE_AHRF_STAGE'
FILE_FORMAT = 'CSV_FORMAT'
DATABASE = 'JBS_DATASETS'
SCHEMA = 'AREA_HEALTH_RESOURCE_FILE_AHRF'

# === Helpers ===
def clean_name(s):
    s = re.sub(r"[^\w]+", "_", s)       # replace non-alphanum with underscore
    return re.sub(r'_+', '_', s).strip('_').upper()

def clean_filename(file_path, sheet=None):
    base = os.path.splitext(os.path.basename(file_path))[0]
    if sheet: base += f"_{sheet}"
    return clean_name(base) + '.csv'

def process_df(df, path):
    df.columns = [clean_name(c) for c in df.columns]
    #df.to_csv(path, index=False)
    return df

# === Process files ===
cleaned = []

for f in files:
    ext = os.path.splitext(f)[1].lower()
    if ext in ['.csv', '.tsv']:
        df = pd.read_csv(f, sep=',' if ext=='.csv' else '\t', low_memory=False)
        out = os.path.splitext(f)[0] + "_cleaned.csv"
        process_df(df, out)
        cleaned.append((out, clean_filename(out)))
    elif ext == '.xlsx':
        xls = pd.ExcelFile(f)
        for sh in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sh)
            sh = clean_name(sh)
            out = os.path.splitext(f)[0] + f"_{sh}_cleaned.csv"
            process_df(df, out)
            cleaned.append((out, clean_filename(f, sh)))

# === Snowflake PUT statements ===
print("\n-- PUT statements --")
for local, stage_file in cleaned:
    path = os.path.abspath(local).replace("\\","/")
    print(f'snow sql -q "PUT \'file://{path}\' {STAGE}/{stage_file} AUTO_COMPRESS=FALSE OVERWRITE=TRUE;"')

# === CREATE TABLE + COPY INTO ===
print("\n-- CREATE TABLE & COPY INTO --")

for local, tbl_full in cleaned:
    df = pd.read_csv(local, nrows=0)
    cols = ",\n    ".join([f'"{c}" VARCHAR' for c in df.columns])
    tbl = tbl_full.removesuffix(".csv")
    create = f"CREATE OR REPLACE TABLE {DATABASE}.{SCHEMA}.{tbl} (\n    {cols}\n);"
    copy = f"""
COPY INTO {DATABASE}.{SCHEMA}.{tbl}
FROM {STAGE}/{tbl_full}
FILE_FORMAT = (FORMAT_NAME = {FILE_FORMAT})
ON_ERROR = 'CONTINUE';
""".strip()
    print(f"\n-- {tbl} --\n{create}\n{copy}\n")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 122286: invalid continuation byte

In [8]:
import pandas as pd
import os
import re

# === CONFIG ===





files_og = [
    r"C:\Users\apagano\Downloads\srei_zcta_v101723.xlsx",
    r"C:\Users\apagano\Downloads\srei_v08042023.xlsx",
]



files = [
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024hf.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024hp.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023HP.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023POP.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023UTL.csv",
    r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023ENV.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023EXP.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023GEO.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023HF.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrfsn2023.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrfsn2024.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrfsn2025.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025pop.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025utl.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025env.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025exp.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025geo.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025hf.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\AHRF2025hp.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024pop.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024utl.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024_Feb2025.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024env.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024exp.csv",
r"C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024geo_Feb2025.csv",]











filestesting = [

    r"...\Agreement Testing_Empl.xlsx",
]
STAGE = '@JBS_DATASETS.AREA_HEALTH_RESOURCE_FILE_AHRF.AREA_HEALTH_RESOURCE_FILE_AHRF_STAGE'
FILE_FORMAT = 'CSV_FORMAT'
DATABASE = 'JBS_DATASETS'
SCHEMA = 'AREA_HEALTH_RESOURCE_FILE_AHRF'

# === Helpers ===
def clean_name(s):
    s = re.sub(r"[^\w]+", "_", s)       # replace non-alphanum with underscore
    return re.sub(r'_+', '_', s).strip('_').upper()

def clean_filename(file_path, sheet=None):
    base = os.path.splitext(os.path.basename(file_path))[0]
    if sheet: base += f"_{sheet}"
    return clean_name(base) + '.csv'

def process_df(df, path):
    df.columns = [clean_name(c) for c in df.columns]
    df.to_csv(path, index=False)
    return df

# === Process files ===
cleaned = []

for f in files:
    ext = os.path.splitext(f)[1].lower()
    
    # --- Process CSV / TSV files ---
    if ext in ['.csv', '.tsv']:
        sep = ',' if ext == '.csv' else '\t'
        try:
            df = pd.read_csv(f, sep=sep, low_memory=False)
        except UnicodeDecodeError:
            # Fallback to latin1 encoding if UTF-8 fails
            df = pd.read_csv(f, sep=sep, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {f}: {e}")
            continue
        
        out = os.path.splitext(f)[0] + "_cleaned.csv"
        process_df(df, out)
        cleaned.append((out, clean_filename(out)))
        print(f"Processed CSV/TSV: {f}")
    
    # --- Process XLSX files ---
    elif ext == '.xlsx':
        try:
            xls = pd.ExcelFile(f)
        except Exception as e:
            print(f"Error opening Excel file {f}: {e}")
            continue
        
        for sh in xls.sheet_names:
            try:
                df = pd.read_excel(xls, sheet_name=sh)
            except Exception as e:
                print(f"Error reading sheet {sh} in {f}: {e}")
                continue
            
            clean_sh = clean_name(sh)
            out = os.path.splitext(f)[0] + f"_{clean_sh}_cleaned.csv"
            process_df(df, out)
            cleaned.append((out, clean_filename(f, clean_sh)))
            print(f"Processed sheet '{sh}' in Excel: {f}")


# === Snowflake PUT statements ===
print("\n-- PUT statements --")
for local, stage_file in cleaned:
    path = os.path.abspath(local).replace("\\","/")
    print(f'snow sql -q "PUT \'file://{path}\' {STAGE}/{stage_file} AUTO_COMPRESS=FALSE OVERWRITE=TRUE;"')

# === CREATE TABLE + COPY INTO ===
print("\n-- CREATE TABLE & COPY INTO --")

for local, tbl_full in cleaned:
    df = pd.read_csv(local, nrows=0)
    cols = ",\n    ".join([f'"{c}" VARCHAR' for c in df.columns])
    tbl = tbl_full.removesuffix(".csv")
    create = f"CREATE OR REPLACE TABLE {DATABASE}.{SCHEMA}.{tbl} (\n    {cols}\n);"
    copy = f"""
COPY INTO {DATABASE}.{SCHEMA}.{tbl}
FROM {STAGE}/{tbl_full}
FILE_FORMAT = (FORMAT_NAME = {FILE_FORMAT})
ON_ERROR = 'CONTINUE';
""".strip()
    print(f"\n-- {tbl} --\n{create}\n{copy}\n")


Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024hf.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2024hp.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023HP.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023POP.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023UTL.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023ENV.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023EXP.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023GEO.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrf2023HF.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\datafiles\ahrfsn2023.csv
Processed CSV/TSV: C:\Users\apagano\Downloads\AHRF\csv_xlsx\da

In [4]:
import shutil
from pathlib import Path

def move_csv_xlsx_files(input_path):
    input_path = Path(input_path)
    
    if not input_path.exists():
        print(f"The path {input_path} does not exist.")
        return
    
    # Create the destination folder next to input_path
    dest_folder = input_path.parent / "csv_xlsx"
    dest_folder.mkdir(exist_ok=True)
    
    # Walk recursively through the input path
    for file_path in input_path.rglob("*"):
        if file_path.is_file() and file_path.suffix.lower() in [".csv", ".xlsx"]:
            try:
                shutil.move(str(file_path), dest_folder / file_path.name)
                print(f"Moved: {file_path} -> {dest_folder / file_path.name}")
            except Exception as e:
                print(f"Error moving {file_path}: {e}")

# Example usage
input_folder = r"C:\Users\apagano\Downloads\AHRF\Extracted"
move_csv_xlsx_files(input_folder)


Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\ahrf2024_Feb2025.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024_Feb2025.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\CSV Files by Categories\ahrf2024env.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024env.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\CSV Files by Categories\ahrf2024exp.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024exp.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\CSV Files by Categories\ahrf2024geo_Feb2025.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024geo_Feb2025.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\CSV Files by Categories\ahrf2024hf.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024hf.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 C

In [5]:
import shutil
from pathlib import Path

def move_other_files(input_path):
    input_path = Path(input_path)
    
    if not input_path.exists():
        print(f"The path {input_path} does not exist.")
        return
    
    # Create the destination folder next to input_path
    dest_folder = input_path.parent / "other_files"
    dest_folder.mkdir(exist_ok=True)
    
    # Walk recursively through the input path
    for file_path in input_path.rglob("*"):
        if file_path.is_file() and file_path.suffix.lower() not in [".csv", ".xlsx"]:
            try:
                shutil.move(str(file_path), dest_folder / file_path.name)
                print(f"Moved: {file_path} -> {dest_folder / file_path.name}")
            except Exception as e:
                print(f"Error moving {file_path}: {e}")

# Example usage
input_folder = r"C:\Users\apagano\Downloads\AHRF\Extracted"
move_other_files(input_folder)


Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\2023-2024_AHRFDUA.docx -> C:\Users\apagano\Downloads\AHRF\other_files\2023-2024_AHRFDUA.docx
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 User Tech\AHRF 2023-2024 User Tech\AHRF USER 2023-2024_Feb2025.docx -> C:\Users\apagano\Downloads\AHRF\other_files\AHRF USER 2023-2024_Feb2025.docx
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 User Tech\AHRF 2023-2024 User Tech\What's New 2023-2024 AHRF_Feb2025.pdf -> C:\Users\apagano\Downloads\AHRF\other_files\What's New 2023-2024 AHRF_Feb2025.pdf
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF SN 2023-2024 CSV\AHRF SN 2023-2024 CSV\2023-2024_AHRFDUA.docx -> C:\Users\apagano\Downloads\AHRF\other_files\2023-2024_AHRFDUA.docx
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF SN 2023-2024 User Tech\AHRF SN 2023-2024 User Tech\AHRF SN User 2023-2024.docx -> C:\Users\apagano\Downloads\AHRF\other_files\AHRF SN User 2023-20